**RECONOCIMIENTO Y SÍNTESIS DE VOZ**

*John Atkinson*

Este programa utiliza servicios pre-entrenados de Google para *reconocimiento automático de  voz* (ASR) y *síntesis de texto a voz* (TTS) en Español.

*   El módulo de ASR utiliza el servicio [SpeechRecognition](https://realpython.com/python-speech-recognition/). Note que *Google Colab* no puede procesar el dispositivo de audio directamente (i.e., micrófono) por lo que se realizó unos pequeños ajustes invocando a rutinas en JavaScript.

*   El módulo de TTS utiliza el servicio [gTTS](https://gtts.readthedocs.io/en/latest/module.html).

Primero, necesitamos instalar algunos paquetes:

In [ ]:
apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
pip install speechrecognition
pip install PyAudio
pip -q install pydub
pip install gtts
pip install pygobject

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

     |████████████████████████████████| 32.8 MB 239 kB/s eta 0:00:011     |▊                               | 778 kB 3.3 MB/s eta 0:00:10     |█▋                              | 1.6 MB 3.3 MB/s eta 0:00:10     |███▏                            | 3.3 MB 3.3 MB/s eta 0:00:09     |█████████████████████▉          | 22.4 MB 48.8 MB/s eta 0:00:01     |██████████████████████▋         | 23.2 MB 48.8 MB/s eta 0:00:01     |███████████████████████▌        | 24.1 MB 48.8 MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /Users/eduardo/.local/share/virtualenvs/eduardo-BbA8MaND/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/k_/5bn7s8bj3q50cjmfsm__qhlr0000gn/T/pip-install-ciehy0w0/pyaudio_ede4e193defb45eb8befd6aa3c4a2d50/setup.py'"'"'; __file__='"'"'/private/var/folder

Luego, importamos algunas bibliotecas para manejo de audio, reconocimient y síntesis:

In [ ]:
import speech_recognition as sr 
from pydub import AudioSegment
from IPython.display import Javascript, Audio
from google.colab import output
from base64 import b64decode
from io import BytesIO
import subprocess
import re
from gtts import gTTS 

Dado que nuestro programa está corriendo en *Google Colab*, el manejo de audio no es tan directo como cuando el programa se encuentra en nuestro propio computador ya que accede directamente a los dispositivos de audio (micrófono, parlantes). 

Para resolver este problema, realizamos dos pasos:
1. Grabar el audio y almacenarlo en un archivo de sonido (se debe utilizar *JavaScript*).
2. Realizar el reconocimiento de voz a partir del audio previamente grabado.

Para esto,  definimos la función **Grabar()**, que graba un audio utilizando un código JavaScript en el archivo temporal "*out00.wav*":

In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def Grabar(sec=5):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.webm','wb') as f:
    f.write(b)
  command = ['ffmpeg', '-i', 'audio.webm', '-f', 'segment', '-segment_time', '100', 'out%02d.wav']
  subprocess.run(command,stdout=subprocess.PIPE,stdin=subprocess.PIPE)
  return('out00.wav')

Definimos la función **ASR()** que realiza el reconocimiento de voz propiamente tal, y que utiliza el módulo de grabado de audio, especificado previamente:

In [ ]:
def ASR():
    texto_reconocido=""
    print("Comience a hablar..")
    # Si ejecutará el programa en su computador (NO Colab), simplemente
    # reemplace las líneas desde "WAVES=.."  hasta "try:" por las siguientes:
    #r = sr.Recognizer()                                                                                   
    #with sr.Microphone() as source:                                                                     
    #   print(".")
    #   audio = r.listen(source)   
    #   print("..")
    #try: 
    WAVES = Grabar()
    fuente = sr.AudioFile(WAVES)
    r = sr.Recognizer()
    with fuente as source:
         audio = r.record(source)                                 
    try:
         texto_reconocido = r.recognize_google(audio, language='es-es')
    except sr.UnknownValueError:
       print("No entendi el audio")
    return(texto_reconocido)

Luego,  definimos la función  **TTS(texto)**, que realiza la síntesis de voz en Español utilizando el método **gTTS(..)**,  a partir de un **texto**, utilizando modelos *fonéticos* pre-entrenados:

In [ ]:
def TTS(texto):
  sound_file = 'tmp.wav'
  tts = gTTS(texto, lang="es")
  tts.save(sound_file) 
  display(Audio(sound_file,autoplay=True))  

Finalmente, hacemos una prueba del ASR y TTS desde el programa principal. Para esto, buscamos si en la voz reconocida del usuario aparece la palabra "*auto*" en algún lugar:

In [ ]:
texto = ASR()
print("Dijiste: ",texto)
calce = re.search("auto",texto)
if calce != None:
    TTS("Usted necesita comprar un auto..")